In [75]:
import torch
import numpy as np

In [76]:
# tensor 和 numpy 对象的相互转化
# a = np.array([[1, 2, 3], [4, 5, 6]])
# b = torch.tensor(a)
# c = b.numpy()
# type(a), type(b), type(c)

In [77]:
# 只有一个元素的向量可以转化为标量
# a = torch.tensor([3.5])
# a, a.item(), float(a), int(a)

In [78]:
# # reshape的一些特性
# a = torch.arange(12)
# b = a.reshape(3, 4)
# b[:] = 2    # 修改所有的值
# a, b   # 地址没有改变，实在原地址上进行操作，但是形状又不一样

In [79]:
import sympy as sp
import numpy as np
sp.init_printing(use_unicode=True)

In [80]:
X = np.array([[3, 3], [4, 3], [1, 1]])
y = np.array([1, 1, -1])
# 通过样本数量创建相应的拉格朗日乘子
alphas = sp.Matrix([sp.symbols('alpha_0:%d' % X.shape[0])])
weights = sp.Matrix([sp.symbols('w_0:%d' % X.shape[1])])
b = sp.symbols('b')

# n = sp.Idx('i', (1, X.shape[0]))
# n, i, j = sp.symbols('n i j', integer=True)

# 构建目标函数
objective_func = np.sum([weight**2 for weight in weights]) / 2
# objective_func = sp.Sum([weight**2 for weight in weights], (i, 1, n)) / 2

# 构建约束函数
subject_to_func = sp.Matrix([1-yi*(np.sum(weights*Xi)+b) for yi, Xi in zip(y, X)])
# 构建拉格朗日函数
lagrange_func = objective_func + np.sum(alphas*subject_to_func)
# sp.diff(lagrange_func, weights[0])
diff_w = sp.Matrix(sp.diff(lagrange_func, weights))
# diff_w, lagrange_func, subject_to_func
# alphas.T
diff_b = sp.diff(lagrange_func, b)
lagrange_func
# np.sum(alphas*subject_to_func)
# sp.Matrix(sp.diff(lagrange_func, weights))
weights.shape

In [81]:
alpha_st_kkts = alphas.T * subject_to_func.T
# alpha_st_kkts.shape
# a = [item for item in sp.Matrix(alpha_st_kkts)]
# a = np.array([item for item in sp.Matrix(alpha_st_kkts)]).reshape(3, 3)

alpha_st_kkts = sp.Matrix(np.diag(np.array(alphas.T * subject_to_func.T)))
# alphas*subject_to_func.T
# subject_to_func
# [alphas.T * subject_to_func.T][0].shape
# subject_to_func.T
alpha_st_kkts

⎡α₀⋅(-b - 3⋅w₀ - 3⋅w₁ + 1)⎤
⎢                         ⎥
⎢α₁⋅(-b - 4⋅w₀ - 3⋅w₁ + 1)⎥
⎢                         ⎥
⎣  α₂⋅(b + w₀ + w₁ + 1)   ⎦

In [82]:
alpha_kkts = sp.Matrix([sp.Ge(alpha, 0) for alpha in alphas])
alpha_kkts

⎡α₀ ≥ 0⎤
⎢      ⎥
⎢α₁ ≥ 0⎥
⎢      ⎥
⎣α₂ ≥ 0⎦

In [83]:
st_kkts = sp.Matrix([sp.Le(st, 0) for st in subject_to_func])
st_kkts

⎡-b - 3⋅w₀ - 3⋅w₁ + 1 ≤ 0⎤
⎢                        ⎥
⎢-b - 4⋅w₀ - 3⋅w₁ + 1 ≤ 0⎥
⎢                        ⎥
⎣  b + w₀ + w₁ + 1 ≤ 0   ⎦

In [84]:
solve = sp.solve([*diff_w, diff_b, *alpha_st_kkts], [*weights, b, *alphas])
# [*diff_w, diff_b, *alpha_st_kkts]
# [*weights, b, *alphas]
result = sp.Matrix(solve)
w = result[:, 0:weights.shape[1]]
a = result[:, 3:result.shape[1]]
b = result[:, weights.shape[1]]
# b
result
b

⎡ -1 ⎤
⎢    ⎥
⎢ b  ⎥
⎢    ⎥
⎢ -2 ⎥
⎢    ⎥
⎢-23 ⎥
⎢────⎥
⎢ 13 ⎥
⎢    ⎥
⎣ -2 ⎦

In [85]:
result
# for i in b:
#     for j, k in zip(w, a):
#             # dic = {'w': i, 'b': j, 'a': k}
#             # print(dic)
#             print(j, i)
# w[3, :]
# a[3, :]
# w[i, :], b[i], a[i, :]
# sp.Matrix([*w[0, :], b[0], *a[0, :]]).T
# for i in range(b.shape[0]):
#     print(w[i, :], b[i], a[i, :])
# [*w[0, :], b[0], *a[0, :]]
# sp.Matrix([[*w[i, :], b[i], *a[i, :]] for i in range(b.shape[0])])
# temp = [[w[i, :], b[i], a[i, :]] for i in range(b.shape[0])]
# [{'w': j[0], 'b': j[1], 'a': j[2]} for j in [[w[i, :], b[i], a[i, :]] for i in range(b.shape[0])]]
    # dic = {'w': j[0], 'b': j[1], 'a': j[2]}
    # print(dic)
r = sp.Matrix([{'w': j[0], 'b': j[1], 'a': j[2]} for j in [[w[i, :], b[i], a[i, :]] for i in range(b.shape[0])]])
t1 = sp.Matrix([*r[4]['a'], r[4]['b'], *r[4]['w']])
# r[0]
# t1.T

# r
# r[0]['a'][1]
# for i in range(3):
#     a = alpha_kkts[i].subs(alphas, r[0]['a'])
#     type(a)

# for i in range(3):
#     t = st_kkts[i].subs(subject_to_func[i], [*r[0]['b'], *r[0]['w'][i]])
#     print(t)

# for i in range(r.shape[0]):
#     for j in range(3):
#         t = st_kkts[i].subs(subject_to_func[i], [*r[0]['b'], *r[0]['w'][i]])
#         print(t)
alpha_kkts[0].subs(alphas[0], r[0]['a'][0])
r[0]['b']
t1.T

[1/4  0  1/4  -2  1/2  1/2]

In [86]:
# for i in range(alpha_kkts.shape[0]):
# if alpha_kkts[i].subs(r[0]['a']):
#     print('yes')
# alpha_kkts.shape[0]

# 将结果转化为字典
for i in range(r.shape[0]):
    param_dict = dict()
    for j in range(alphas.shape[1]):
        param_dict[str(alphas[j])] = r[i]['a'][j]    
    param_dict['b'] = r[i]['b']    
    for j in range(weights.shape[1]):
        param_dict[str(weights[j])] = r[i]['w'][j]
    

# [d.update({alphas[i] : r[0]['a'][i]}) for i in range(3)]
# [{alphas[i] : r[0]['a'][i]} for i in range(3)]
# a = [{alphas[i] : r[0]['a'][i]} for i in range(3)]
# for i in range(alphas.shape[1]):
#     d[alphas[i]] = r[0]['a'][i]
# print(d)
    # d[alphas[i]] = r[0]['a'][i]
# print(d)

In [121]:
# for i in range(alpha_kkts.shape[0]):
#     if alpha_kkts[i].subs(alphas[i], r[0]['a'][i])
#         print('yes')
alpha_kkts.shape[0]
# alpha_kkts[i].subs(r[0]['a'][i])
# r[2]['a'][0]
print(param_dict)
# if alpha_kkts[0].subs(param_dict)
# alpha_kkts[0].subs(alphas[0], 1)
alpha_kkts[0].subs(param_dict)

test = {'alpha_0': 1/4, 'alpha_1': 0, 'alpha_2': -1, 'b': -2, 'w_0': 1/2, 'w_1': 1/2}
# for i in range(alpha_kkts.shape[0]):
#     if not alpha_kkts[i].subs(param_dict):
#         continue
[i for i in range(alphas.shape[0]) if not alpha_kkts[i].subs(test)]
# alpha_kkts.shape
# for j in range(st_kkts.shape[0]):
#     if not st_kkts[i].subs(param_dict):
#         continue
# st_kkts[2].subs(param_dict)

{'alpha_0': 1/4, 'alpha_1': 0, 'alpha_2': 1/4, 'b': -2, 'w_0': 1/2, 'w_1': 1/2}


In [67]:
# from sympy.plotting import plot3d
# x, y = sp.symbols('x, y')
# plot3d(x*sp.exp(-x**2 - y**2), (x, -3, 3), (y, -2, 2))
# formula = x*sp.exp(-x**2 - y**2)
# formula

In [68]:
# import sympy.stats as stat
# a = sp.Matrix(sp.symbols(' '.join([('a_%d' % i) for i in range(10)])))
# y = sum([a[m]*x**m for m in range(10)])
# sp.summation(a[0], (j, 1, i))
# sp.Matrix(sp.symbols('a_0:%d' % 10)).T